In [1]:
# MLP with automatic validation set
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics

Using TensorFlow backend.


In [2]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

In [3]:
def purity_score_per_cluster(data, N):
    return (data['y'].value_counts().max() / N)

In [4]:
# load pima indians dataset
#dataset = numpy.loadtxt("D:/Neural_nets_course_Dataset/pima_indians_diabetes_data.csv", delimiter=",")
dataset = pd.read_csv('E:/Academic/Neural Networks/Assignment_2/iris.txt', delimiter=',', names = ['a','b','c','d','y'])

In [8]:
dataset.head()

,a,b,c,d,y
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.6,3.1,1.5,0.2,1
3,5.0,3.6,1.4,0.2,1
4,4.6,3.4,1.4,0.3,1


In [5]:
X = dataset.iloc[:,0:4].values
Y = dataset.iloc[:,4].values

In [18]:
# initilaize to very small random values
weight_matrix = np.random.rand(3, 4)
#weight_matrix_norma = preprocessing.normalize(weight_matrix, norm='l2')

In [19]:
weight_matrix

array([[0.84175922, 0.30500036, 0.48831798, 0.88720764],
       [0.87403121, 0.72834783, 0.14212033, 0.70981784],
       [0.58188005, 0.95242927, 0.77550592, 0.99208439]])

In [20]:
X = np.array(X)

In [21]:
cluster_indexes = []
alpha = 0.3

for i in range(500):
    cluster_indexes = []
    for index,iterator in enumerate(X):
        # calculate the distance vector
        out_neuron_vector = np.linalg.norm(weight_matrix - X[index], axis=1, ord=2)
        min_index = np.argmin(out_neuron_vector)
        # add cluster value for each tuple
        cluster_indexes.append(min_index + 1)
        # retrieve Wk vector
        Wk = weight_matrix[min_index,:]
        # update the Wk vector
        pwx = alpha * np.subtract(X[index], Wk)
        Wk = np.add(Wk, pwx)
        # add Wk to original weight matrix location
        weight_matrix[min_index,:] = Wk

In [22]:
np.unique(cluster_indexes)

array([2, 3], dtype=int64)

In [23]:
len(cluster_indexes)

150

In [24]:
temp_dataset = dataset
temp_dataset['cluster'] = cluster_indexes

purity_score_cluster1 = 0.0
purity_score_cluster2 = 0.0
purity_score_cluster3 = 0.0

#purity score for cluster 1
cluster1_dataset = temp_dataset[temp_dataset['cluster'] == 1]
if len(cluster1_dataset) != 0:
    purity_score_cluster1 = purity_score_per_cluster(cluster1_dataset, len(dataset))

#purity score for cluster 2
cluster2_dataset = temp_dataset[temp_dataset['cluster'] == 2]
if len(cluster2_dataset) != 0:
    purity_score_cluster2 = purity_score_per_cluster(cluster2_dataset, len(dataset))

#purity score for cluster 3
cluster3_dataset = temp_dataset[temp_dataset['cluster'] == 3]
if len(cluster3_dataset) != 0:
    purity_score_cluster3 = purity_score_per_cluster(cluster3_dataset, len(dataset))

print('Purity score for cluster 1: ', purity_score_cluster1)
print('Purity score for cluster 2: ', purity_score_cluster2)
print('Purity score for cluster 3: ', purity_score_cluster3)
print('Total purity score: ', purity_score(Y, cluster_indexes))

Purity score for cluster 1:  0.0
Purity score for cluster 2:  0.3333333333333333
Purity score for cluster 3:  0.3333333333333333
Total purity score:  0.6666666666666666
